# Complexidade de letras:

## Introdução:

Este código foi desenvolvido com o intuito de explorar e revelar a complexidade das letras musicais, proporcionando uma análise detalhada de sua riqueza lírica. Diversos critérios foram considerados para avaliar a profundidade e sofisticação das composições, incluindo a diversidade temática das letras. A seguir, estão os principais critérios utilizados na análise:

Número de sílabas: Palavras com mais sílabas geralmente indicam maior complexidade, tanto em termos de compreensão quanto de pronúncia.

Frequência de palavras: A frequência com que as palavras aparecem na letra pode sugerir sua acessibilidade e familiaridade para o público.

Análise gramatical: Frases com muitos gerúndios tendem a ser mais coloquiais, enquanto aquelas com maior uso de conjunções subordinadas indicam um tom mais formal.

Diversidade lexical: A variedade de vocabulário e a riqueza semântica contribuem significativamente para a expressividade e profundidade das letras.

Diversidade temática: A amplitude dos temas abordados nas letras reflete sua complexidade, com composições que exploram uma gama mais ampla de assuntos sendo consideradas mais ricas e elaboradas.

##  2 - Desenvolvimento:

### 2.1 - Importando bibliotecas:

In [67]:
import pandas as pd
import textstat as tst
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy as np

### 2.2 - Leitura do dataset:

In [68]:
df = pd.read_csv("dataset_atualizado.csv")

### 2.3 - Descrição do dataset utilizado:

In [69]:
df.head()

,artist,track_name,lyrics,genre
0,abba,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA...",pop
1,abba,"Andante, Andante","Take it easy with me, please \r\nTouch me gen...",pop
2,abba,As Good As New,I'll never know why I had to go \r\nWhy I had...,pop
3,abba,Bang,Making somebody happy is a question of give an...,pop
4,abba,Bang-A-Boomerang,Making somebody happy is a question of give an...,pop


### 2.4 - Limpeza de dados:

Como são letras de música, as linhas contêm \r ou \n, o que, futuramente, pode ocasionar problemas de entendimento, sendo necessário filtrá-las

In [70]:
def limpar_texto(texto):
    texto = texto.replace("\r\n", " ").replace("\n", " ").strip()
    return texto
    
df['letras_limpas'] = df['lyrics'].apply(limpar_texto)

## 3 - Análises exploratórias:

### 3.1 - Número de sílabas:

Este código tem como objetivo contar o número de sílabas de cada palavra em letras de músicas e identificar palavras que possuem uma certa complexidade lírica.

### Passo 1: Criar um Dicionário de Sílaba

Primeiro, criamos um dicionário que armazena o número de sílabas de cada palavra em cada música. O uso do índice é necessário para evitar que músicas com nomes iguais recebam o mesmo número de sílabas.

In [71]:
# Cria um dicionário para armazenar o número de sílabas de cada palavra em cada música.
# O índice é usado para evitar que músicas com nomes iguais recebam o mesmo número de sílabas.
dic_silabas = {df.track_name[n]: {} for n in range(len(df))} 

for n in range(len(df)):
    lista = df.letras_limpas[n].split(" ")
    for palavra in lista:
        # Se a palavra ainda não estiver no dicionário da música, conta as sílabas
        if palavra not in dic_silabas[df.track_name[n]]:
            dic_silabas[df.track_name[n]][palavra] = tst.syllable_count(palavra)

### Passo 2: Contar Palavras Complexas

Aqui, percorremos as músicas e suas palavras para identificar palavras que possuem 4 ou mais sílabas. Também filtramos expressões repetitivas para uma análise mais precisa.

In [72]:
# Dicionário para armazenar a contagem de palavras com 4 ou mais sílabas em cada música
musicas_silabas_complexas = {}

for nome_musica, palavras in dic_silabas.items():
    for palavra, silabas in palavras.items():
        # Verifica se a palavra tem 4 ou mais sílabas
        if silabas >= 4:
            # Filtra palavras que contêm uma letra quatro ou mais vezes (ex.: "Ahhh"), comum em músicas
            if not any(palavra.count(letra) >= 4 for letra in set(palavra)):
                if nome_musica not in musicas_silabas_complexas:
                    musicas_silabas_complexas[nome_musica] = 1
                else:
                    musicas_silabas_complexas[nome_musica] += 1

### Passo 3: Criar uma coluna no dataset

Aqui, criaremos uma coluna referente a quantidade de sílabas maiores que 4 que a letra possui

In [73]:
df['Silabas'] = df['track_name'].map(musicas_silabas_complexas).fillna(0)

### 3.2 - Frequência:

Este código visa calcular a diversidade lexical das letras das músicas, utilizando a medida TTR (Type-Token Ratio), que é a razão entre o número de palavras únicas e o total de palavras em uma letra. 

### Passo 1: Contar Palavras Únicas

Nesta etapa, percorremos as letras das músicas, convertendo-as para minúsculas e dividindo-as em palavras. Em seguida, contamos o número de palavras únicas em cada letra e armazenamos esses valores em uma lista.

In [74]:
palavras_unicas_list = []  # Lista para armazenar o número de palavras únicas
lista_ttr = []              # Lista para armazenar os valores TTR

for letra in range(len(df)):
    palavras = df.letras_limpas[letra].lower().split() 

    # Adiciona o número de palavras únicas à lista
    palavras_unicas_list.append(len(set(palavras)))      

### Passo 2: Calcular o TTR (Type-Token Ratio)

Aqui, calculamos o TTR para cada letra, dividindo o número de palavras únicas pelo total de palavras na letra. O resultado é armazenado em uma lista para análise posterior.

In [75]:
# Calcula o TTR para cada letra
for n in range(len(palavras_unicas_list)):
    lista_ttr.append(palavras_unicas_list[n] / len(df.lyrics[n])) 

### Passo 3: Criar uma coluna no dataset

In [76]:
df["Diversidade lexical"] = lista_ttr

### 3.3 - Análise Gramatical:

Este código tem como objetivo calcular uma pontuação com base nas tags gramaticais das palavras presentes nas letras das músicas. A pontuação é atribuída de acordo com a presença de substantivos, adjetivos, advérbios, verbos e outras categorias gramaticais. Para realizar a análise gramatical, utilizamos a biblioteca NLTK (Natural Language Toolkit), que fornece ferramentas para a tokenização e a análise de partes do discurso (POS tagging).

### Passo 1: Inicializar o Dicionário de Pontuação

Primeiramente, criamos um dicionário (`dic_tags`) onde cada música é uma chave e sua pontuação inicial é zero. Isso nos permitirá acumular a pontuação ao longo da análise.

In [77]:
dic_tags = {track: 0 for track in df['track_name']}  # Inicializa o dicionário de pontuação para cada música

### Passo 2: Iterar Sobre as Letras e Calcular a Pontuação

Neste passo, utilizamos um loop para percorrer cada letra do DataFrame. Para cada letra, realizamos a tokenização e a análise de partes do discurso (POS tagging) para identificar a categoria gramatical de cada palavra. Em seguida, atribuímos uma pontuação com base nas tags gramaticais.

In [78]:
for index, row in df.iterrows():
    tokens = word_tokenize(row['lyrics'].lower())  # Tokeniza a letra em palavras
    tags = nltk.pos_tag(tokens) 
    
    pontuacao = 0  
    for palavra, tag in tags:
        # Atribui pontuação com base nas tags gramaticais
        if tag in ['NNP', 'NNPS']:  # Substantivos próprios
            pontuacao += 0.02
        elif tag in ['JJ', 'JJR', 'JJS']:  # Adjetivos
            pontuacao += 0.01
        elif tag in ['RB', 'RBR', 'RBS']:  # Advérbios
            pontuacao += 0.01
        elif tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:  # Verbos
            pontuacao += 0.01
        elif tag == 'WDT':  # Determinante interrogativo
            pontuacao += 0.01 
        elif tag == 'IN':  # Preposição ou conjunção subordinativa 
            pontuacao += 0.01
            
    dic_tags[row['track_name']] = pontuacao 

### Passo 3: Criar uma coluna no dataset

In [79]:
df['pontuacao'] = df['track_name'].map(dic_tags)

### 3.4 - Diversidade Lexical:

Este trecho de código tem como objetivo calcular a complexidade das letras das músicas usando o modelo Word2Vec. A complexidade é determinada pela variância dos vetores das palavras, permitindo uma análise da riqueza e diversidade vocabular presente nas letras. A variância é utilizada porque indica o grau de dispersão dos vetores em relação à média, refletindo a diversidade vocabular: uma maior variância sugere um vocabulário mais rico e variado, enquanto uma menor variância indica similaridade entre as palavras, resultando em uma letra potencialmente menos complexa.

### Passo 1: Remoção de Palavras Comuns (Stop Words)

Primeiro, criamos um conjunto de palavras comuns (stop words) que serão removidas das letras, uma vez que essas palavras geralmente não contribuem para a variabilidade temática.

In [80]:
stop_words = set(stopwords.words('english')) 

### Passo 2: Preparar lista de palavras para o modelo

O modelo requer que as palavras estejam em listas separadas para facilitar a análise, visto que ele calcula as palavras pela coocorrência

In [81]:
letras = [[letra for letra in df['letras_limpas'][n].split() if letra not in stop_words] for n in range(len(df))]

### Passo 3: Treinamento do Modelo Word2Vec

Em seguida, utilizamos o Word2Vec para criar um modelo que representa as palavras em um espaço vetorial. Isso nos permitirá calcular a complexidade das letras com base nos vetores gerados.

In [82]:
model = Word2Vec(letras, vector_size=100, window=5, min_count=1, workers=4)  # Treina o modelo Word2Vec

### Passo 4: Definição das Funções

#### Passo 4.1 - Cálculo da Variância dos Vetores

A função complexidade_vetores recebe uma lista de palavras e o modelo Word2Vec, retornando a soma da variância dos vetores correspondentes às palavras.

In [83]:
def calcular_variancia_vetores(palavras, modelo):
    vetores = []
    for palavra in palavras:
        if palavra in modelo.wv:
            vetores.append(modelo.wv[palavra])  # Retorna o vetor da palavra
            
    vetores_np = np.array(vetores)  # Converte a lista de vetores em um array NumPy
    variancia = np.var(vetores_np, axis=0)  

    return np.sum(variancia)  

#### Passo 4.2 - Cálculo da Complexidade da Letra

A função calcular_complexidade processa a letra da música e calcula a sua complexidade utilizando a função complexidade_vetores.

In [84]:
def avaliar_complexidade_letra(letra, modelo):
    letra_processada = [palavra.lower() for palavra in letra.split()]  # Processa a letra
    return calcular_variancia_vetores(letra_processada, modelo) 

### Passo 5 - Cálculo da Complexidade para Cada Música

Aqui, inicializamos um dicionário (dic_complexidade) para armazenar a complexidade de cada música e iteramos sobre as letras para calcular essa complexidade.

In [85]:
dic_complexidade = {df.track_name[n]: {} for n in range(len(df))}  # Inicializa o dicionário de complexidade

for index, row in df.iterrows():
    letra = row['letras_limpas']
    complexidade = avaliar_complexidade_letra(letra, model)  
    dic_complexidade[df.track_name[index]] = complexidade  

### Passo 6 - Cálculo da Complexidade das Letras

Calculamos os quartis (Q1 e Q3) e a mediana das complexidades para classificação posterior.

In [86]:
complexidades = [avaliar_complexidade_letra(letra, model) for letra in df['lyrics']]  # Lista de complexidades
q1 = np.percentile(complexidades, 25)  
q3 = np.percentile(complexidades, 75) 
mediana = np.median(complexidades)  

### Passo 7 - Classificação da Variabilidade

Com base nos valores de complexidade, classificamos as músicas em quatro categorias de variabilidade.

In [87]:
for key, value in dic_complexidade.items():
    if value < q1:
        df.loc[df['track_name'] == key, "variabilidade"] = 1  # Baixa variabilidade
    elif q1 < value < mediana:
        df.loc[df['track_name'] == key, "variabilidade"] = 2  # Variabilidade média baixa
    elif mediana < value < q3:
        df.loc[df['track_name'] == key, "variabilidade"] = 3  # Variabilidade média alta
    else:
        df.loc[df['track_name'] == key, "variabilidade"] = 4  # Alta variabilidade

### 3.5 - Diversidade temática

Este código tem como objetivo analisar a presença de temas nas letras das músicas, utilizando um modelo Word2Vec para calcular a similaridade entre os vetores das palavras da letra e os vetores de temas predefinidos. A seguir, descrevemos as principais etapas do processo:

### Passo 1: Definição de Temas e Palavras-chave

In [88]:
themes = {
    'love': ['love', 'affection', 'heart', 'passion', 'desire', 'romance'],
    'sadness': ['sadness', 'pain', 'longing', 'loneliness', 'lament', 'loss'],
    'freedom': ['freedom', 'live', 'free', 'independence', 'choice', 'escape'],
    'friendship': ['friendship', 'companion', 'loyalty', 'trust', 'camaraderie'],
    'hope': ['hope', 'faith', 'dream', 'future', 'light', 'renewal'],
    'nature': ['nature', 'earth', 'sky', 'sea', 'forest', 'wildlife'],
    'struggle': ['struggle', 'battle', 'strength', 'resistance', 'challenge', 'conquest'],
    'joy': ['joy', 'happiness', 'laughter', 'smile', 'celebration', 'fun'],
    'solitude': ['solitude', 'isolation', 'distance', 'disconnection', 'emptiness'],
    'nostalgia': ['nostalgia', 'memories', 'past', 'remembrance', 'reminiscence'],
    'change': ['change', 'transformation', 'growth', 'evolution', 'new beginnings'],
    'self-love': ['self-love', 'self-esteem', 'self-awareness', 'acceptance', 'confidence'],
    'conflict': ['conflict', 'dispute', 'disagreement', 'tension', 'polarization'],
    'solidarity': ['solidarity', 'help', 'support', 'community', 'union'],
    'socio-political': ['socio-political', 'justice', 'rights', 'freedom of speech', 'equality'],
    'party': ['party', 'celebration', 'fun', 'dance', 'joy'],
    'mental health': ['mental health', 'anxiety', 'depression', 'healing', 'balance'],
}

### Passo 2: Cálculo da Média dos Vetores de Palavras

A função media_vetores recebe uma lista de palavras e o modelo Word2Vec, retornando a média dos vetores dessas palavras. Isso permite representar um conjunto de palavras como um único vetor, que captura a essência do significado coletivo.

In [89]:
def media_vetores(palavras, modelo):
    vetores = []
    for palavra in palavras:
        if palavra in modelo.wv:
            vetores.append(modelo.wv[palavra])
    return np.mean(vetores, axis=0) 

### Passo 3: Cálculo da Similaridade Temática

A função resultado é responsável por comparar a letra da música (representada por seu vetor) com os vetores de cada tema. Para cada tema, a função:

a) Calcula o vetor médio das palavras-chave do tema.

Motivação: O vetor médio representa a essência do tema, capturando a semântica das palavras associadas a ele. Isso permite que a análise considere o contexto das palavras, facilitando a comparação com a letra.

b) Compara a similaridade (usando a similaridade cosseno) entre o vetor da letra e o vetor do tema.

Motivação: A similaridade cosseno mede a proximidade entre dois vetores, indicando quão semelhantes são os significados representados. Essa comparação ajuda a identificar a relação temática entre a letra da música e os temas predefinidos.

In [90]:
def resultado(themes, model, vetor_letra):
    # Cria um dicionário para armazenar os resultados de similaridade
    resultado = {}
    for tema, palavras_chave in themes.items():
        # Calcula o vetor médio das palavras-chave do tema usando a função media_vetores
        vetor_tema = media_vetores(palavras_chave, model)
        
        # Verifica se os vetores do tema e da letra não são None
        if vetor_tema is not None and vetor_letra is not None:
            # Calcula a similaridade cosseno entre o vetor da letra e o vetor do tema
            similaridade = model.wv.cosine_similarities(vetor_letra, [vetor_tema])
            
            # Armazena a similaridade do tema no dicionário de resultados
            resultado[tema] = similaridade[0]  # similaridade[0] pega o valor da similaridade
            
    return resultado

### Passo 4: Iteração Sobre as Letras das Músicas

O código itera sobre cada linha do DataFrame df, processando as letras das músicas:

a) Converte as letras para minúsculas e remove palavras de parada (stop words).

b) Calcula o vetor da letra usando a função media_vetores.

c) Chama a função resultado para obter a similaridade entre a letra e os temas, armazenando os resultados em um dicionário chamado dic_div_tematica.

In [91]:
# Cria um dicionário vazio para armazenar a diversidade temática de cada música, usando o nome da música como chave.
dic_div_tematica = {df.track_name[n]: {} for n in range(len(df))}

for index, row in df.iterrows():
    letra = row['letras_limpas'] 
    
    # Processa a letra, convertendo todas as palavras para minúsculas e removendo palavras de parada.
    letra_processada = [palavra.lower() for palavra in letra.split() if palavra.lower() not in stop_words]
    
    vetor_letra = media_vetores(letra_processada, model)
    
    # Obtem o resultado da similaridade temática entre a letra e os temas definidos.
    res = resultado(themes, model, vetor_letra)
    
    dic_div_tematica[df.track_name[index]] = res

### Passo 5: Classificação:

In [92]:
lista_desvios = [np.mean(list(value.values())) for value in dic_div_tematica.values()]

q1 = np.percentile(lista_desvios, 25)  
q3 = np.percentile(lista_desvios, 75) 
mediana = np.median(lista_desvios)  

for key, value in dic_div_tematica.items():
    media_valor = np.mean(list(value.values()))  # Obtém a média dos valores do tema
    if media_valor < q1:
        df.loc[df['track_name'] == key, "diversidade tematica"] = 1  # Baixa diversidade
    elif q1 < media_valor < mediana:
        df.loc[df['track_name'] == key, "diversidade tematica"] = 2  # Diversidade média
    elif mediana < media_valor < q3:
        df.loc[df['track_name'] == key, "diversidade tematica"] = 3  # Alta diversidade
    else:
        df.loc[df['track_name'] == key, "diversidade tematica"] = 4  # Muito alta diversidade

## 4 - Cálculo de complexidade:

In [100]:
pesos = {
    'Diversidade_Lexical': 0.30,
    'Diversidade_Tematica': 0.25,
    'Analise_Gramatical': 0.20,
    'Numero_Silabas': 0.15,
    'Frequentia_Palavras': 0.10
}

df['pontuacao_complexidade'] = (
    df['variabilidade'] * pesos['Diversidade_Lexical'] +
    df['diversidade tematica'] * pesos['Diversidade_Tematica'] +
    df['pontuacao'] * pesos['Analise_Gramatical'] +
    df['Silabas'] * pesos['Numero_Silabas'] +
    df['Diversidade lexical'] * pesos['Frequentia_Palavras']
)

In [117]:
media = df['pontuacao_complexidade'].mean()
desvio_padrao = df['pontuacao_complexidade'].std()

ponto_corte_superior = media + desvio_padrao
ponto_corte_inferior = media - desvio_padrao

df['Complexidade'] = df['pontuacao_complexidade'].apply(lambda x: 1 if x > ponto_corte_superior else (0 if x < ponto_corte_inferior else -1))